<a href="https://colab.research.google.com/github/NoCodeProgram/deepLearning/blob/main/transformer/selfAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NoCodeProgram/deepLearning.git

Cloning into 'deepLearning'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 242 (delta 35), reused 0 (delta 0), pack-reused 139
Receiving objects: 100% (242/242), 12.28 MiB | 20.82 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [3]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Read the text file
with open('deepLearning/transformer/shakespeare.txt', 'r') as file:
    text = file.read()

# Tokenize the text  (this is very simple tokenizer, in reality you would use a more advanced one)
tokenizer = get_tokenizer('basic_english')
tokens = tokenizer(text)
unique_tokens = set(tokens)


In [4]:
stoi = { s:i for i,s in enumerate(unique_tokens)}
itos = { i:s for i,s in enumerate(unique_tokens)}
print(stoi)
print(itos)

vocab_size = len(unique_tokens)
print(vocab_size)

{'eased': 0, 'recured': 1, 'stopped': 2, 'ills': 3, 'perceived': 4, 'hers': 5, 'towards': 6, 'fled': 7, 'vows': 8, 'infection': 9, 'debtor': 10, 'separable': 11, 'cut': 12, 'vainly': 13, 'clay': 14, 'hopes': 15, 'vassal': 16, 'thinking': 17, 'second': 18, 'executor': 19, 'wing': 20, "'": 21, 'filled': 22, 'reckon': 23, 'compared': 24, 'turned': 25, 'impression': 26, 'fierce': 27, 'er-cloying': 28, 'jaws': 29, 'girded': 30, 'patent': 31, 'placed': 32, 'renewed': 33, 'evident': 34, 'glazed': 35, 'coward': 36, 'thought': 37, 'weeds': 38, 'those': 39, 'presage': 40, 'learned': 41, 'seconds': 42, 'successive': 43, 'trees': 44, 'map': 45, 'awake': 46, 'satire': 47, 'subscribes': 48, 'despised': 49, 'laws': 50, 'graced': 51, 'injury': 52, 'shallowest': 53, 'lean': 54, 'attainted': 55, 'tenants': 56, 'eager': 57, 'bitterness': 58, 'hatred': 59, 'sensual': 60, 'masonry': 61, 'fond': 62, 'put': 63, 'pays': 64, 'thriftless': 65, 'cool': 66, 'send': 67, 'pity-wanting': 68, 'little': 69, 'home': 70

In [5]:
sentence = "i love you all"
indices = [stoi[word] for word in sentence.split()]
print(indices)

import torch.nn as nn

embedding_dim = 7
embedding = nn.Embedding(vocab_size, embedding_dim)

embedded_sentence = embedding(torch.tensor(indices))
print(embedded_sentence)


[400, 3032, 2436, 2590]
tensor([[-0.3468, -0.2623, -1.3550, -1.3136,  0.4121,  0.1102, -0.8284],
        [-1.4518,  1.1303,  0.4425, -0.1945, -0.5511,  0.2287, -1.8068],
        [-1.0798,  1.0493, -0.4022,  0.3683,  0.1162,  0.8475, -1.0394],
        [-1.7465, -0.2562,  1.2422, -1.2010, -0.8019,  0.2900, -0.4019]],
       grad_fn=<EmbeddingBackward0>)


In [6]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.query = nn.Linear(embed_dim, head_dim, bias=False)
        self.key = nn.Linear(embed_dim, head_dim, bias=False)
        self.value = nn.Linear(embed_dim, head_dim, bias=False)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)

        print("query shape", query.shape)
        print("key shape", key.shape)
        print("value shape", value.shape)

        scores = torch.matmul(query, key.transpose(-2, -1))
        print("scores shape", scores.shape)
        attention_weights = self.softmax(scores)
        weighted_values = torch.matmul(attention_weights, value)
        print("weighted_values shape", weighted_values.shape)
        return weighted_values


In [7]:
# Create an instance of SelfAttention
head_dim = 5
self_attention = SelfAttention(embedding_dim, head_dim)

output = self_attention(embedded_sentence)
print(output)

query shape torch.Size([4, 5])
key shape torch.Size([4, 5])
value shape torch.Size([4, 5])
scores shape torch.Size([4, 4])
weighted_values shape torch.Size([4, 5])
tensor([[-0.1099, -0.2482,  0.5855, -0.2582,  0.3200],
        [-0.1189, -0.1033,  0.6482, -0.1736,  0.1976],
        [-0.1209, -0.1632,  0.6426, -0.2008,  0.2323],
        [-0.1136,  0.0523,  0.5828, -0.1160,  0.1665]], grad_fn=<MmBackward0>)
